# Train ANN/CNN/RNN Models

In [1]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np

import tensorflow as tf

from model.dataset import SpectraDataset, DataGenerator, DataGeneratorRNN, split2feat

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

2023-01-17 16:18:06.897390: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-17 16:18:07.024991: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-17 16:18:07.586399: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/yoyo/Documents/TensorRT-5.0.2.6/lib
2023-01-17 16:18:07.586451: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_p

## Prepare Dataset

In [2]:
data_dir = 'data'
dataset = SpectraDataset(data_dir)

Reading train data
No. of materials : 22
[['Acrylic' '6']
 ['ENR' '6']
 ['EPDM' '6']
 ['HDPE' '6']
 ['LDPE' '6']
 ['Nylon' '6']
 ['PBAT' '6']
 ['PBS' '6']
 ['PC' '6']
 ['PEEK' '6']
 ['PEI' '6']
 ['PET' '6']
 ['PLA' '6']
 ['PMMA' '6']
 ['POM' '6']
 ['PP' '6']
 ['PS' '6']
 ['PTEE' '6']
 ['PU' '6']
 ['PVA' '6']
 ['PVC' '6']
 ['cellulose' '6']]
Read 132 spectra for training.
No. of data points/shifts per spectra (input shape) : (1792,)
No. of classes : 22
Reading test data
Read 88 spectra for testing.


In [3]:
dataset.get_binary_targets().shape

(132, 22)

In [4]:
dataset.get_training_data().shape

(132, 1792)

In [5]:
datagen = DataGenerator(dataset)

In [6]:
n_features = 24
datagen_rnn = DataGeneratorRNN(dataset, n_features=n_features)

In [7]:
datagen_rnn.data_shape

(132, 75, 24)

## Train

In [8]:
input_shape = dataset.get_input_shape()
num_class = dataset.get_num_class()

input_shape_rnn = datagen_rnn.data_shape

### ANN

In [9]:
from model.nn import ANN

ann = ANN(input_shape, num_class)
ann.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['binary_accuracy', tf.keras.metrics.Recall()])
ann.summary()


ann.fit(datagen, epochs=200, validation_data=dataset.get_test_data())
# ann.fit(x=dataset.get_training_data(), 
#         y=dataset.get_binary_targets(), 
#         epochs=50, 
#         batch_size=32) #optimal

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1792)]            0         
                                                                 
 dense (Dense)               (None, 200)               358600    
                                                                 
 dense_1 (Dense)             (None, 200)               40200     
                                                                 
 dense_2 (Dense)             (None, 22)                4422      
                                                                 
Total params: 403,222
Trainable params: 403,222
Non-trainable params: 0
_________________________________________________________________
Epoch 1/200


2023-01-17 16:18:10.171135: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


4/4 [==============================] - 1s 75ms/step - loss: 0.4311 - binary_accuracy: 0.8281 - recall: 0.1328 - val_loss: 0.2311 - val_binary_accuracy: 0.9545 - val_recall: 0.0000e+00
Epoch 2/200
4/4 [==============================] - 0s 15ms/step - loss: 0.2060 - binary_accuracy: 0.9545 - recall: 0.0000e+00 - val_loss: 0.1849 - val_binary_accuracy: 0.9545 - val_recall: 0.0000e+00
Epoch 3/200
4/4 [==============================] - 0s 11ms/step - loss: 0.1832 - binary_accuracy: 0.9545 - recall: 0.0000e+00 - val_loss: 0.1811 - val_binary_accuracy: 0.9545 - val_recall: 0.0000e+00
Epoch 4/200
4/4 [==============================] - 0s 12ms/step - loss: 0.1822 - binary_accuracy: 0.9545 - recall: 0.0000e+00 - val_loss: 0.1822 - val_binary_accuracy: 0.9545 - val_recall: 0.0000e+00
Epoch 5/200
4/4 [==============================] - 0s 11ms/step - loss: 0.1830 - binary_accuracy: 0.9545 - recall: 0.0000e+00 - val_loss: 0.1809 - val_binary_accuracy: 0.9545 - val_recall: 0.0000e+00
Epoch 6/200
4/4 

### AlexNet

In [10]:
# from model.nn import AlexNet

# alexnet = AlexNet(input_shape, num_class)
# alexnet.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
# alexnet.summary()

# alexnet.fit(datagen, epochs=200)

### VGG16

In [11]:
# from model.nn import VGG16

# vgg16 = VGG16(input_shape, num_class)
# vgg16.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
# vgg16.summary()

# vgg16.fit(datagen, epochs=200, validation_data=dataset.get_test_data())

### ResNet50

In [12]:
# from model.nn import ResNet50

# resnet = ResNet50(input_shape, num_class)
# resnet.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
# resnet.summary()

# resnet.fit(datagen, epochs=20, validation_data=dataset.get_test_data())

### DenseNet

In [13]:
# from model.nn import DenseNet

# densenet = DenseNet(input_shape, num_class)
# densenet.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
# densenet.summary()

# densenet.fit(datagen, epochs=20, validation_data=dataset.get_test_data())

### LSTM

In [14]:
# X_test, y_test = dataset.get_test_data()
# X_test = split2feat(X_test, n_features)

In [15]:
# from model.rnn import VanLSTM

# lstm = VanLSTM(input_shape_rnn, num_class)
# lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
# lstm.summary()

# lstm.fit(datagen_rnn, epochs=20, validation_data=(X_test,y_test))

## Test

In [16]:
from utils import multilabel_classification_report

In [17]:
multilabel_classification_report(dataset, ann)

3/3 [==============================] - 0s 2ms/step - loss: 0.0331 - binary_accuracy: 0.9923 - recall: 0.8409
74 88
            precision     recall   f1-score    support
Acrylic          1.00       0.75       0.86          4
ENR              1.00       1.00       1.00          4
EPDM             1.00       1.00       1.00          4
HDPE             1.00       1.00       1.00          4
LDPE             1.00       0.50       0.67          4
Nylon            1.00       1.00       1.00          4
PBAT             1.00       1.00       1.00          4
PBS              1.00       1.00       1.00          4
PC               1.00       1.00       1.00          4
PEEK             1.00       0.75       0.86          4
PEI              1.00       0.75       0.86          4
PET              1.00       0.50       0.67          4
PLA              1.00       1.00       1.00          4
PMMA             1.00       1.00       1.00          4
POM              1.00       0.25       0.40          4
PP   

In [18]:
# multilabel_classification_report(dataset, alexnet)

In [19]:
# multilabel_classification_report(dataset, vgg16)

In [20]:
# multilabel_classification_report(dataset, resnet)

In [21]:
# multilabel_classification_report(dataset, densenet)

In [22]:
# multilabel_classification_report(dataset, lstm, n_features)

## Predict Unknown Spectra

In [23]:
from model.dataset import UnknownSpectraDataset

In [24]:
dataset.class_name

array(['Acrylic', 'ENR', 'EPDM', 'HDPE', 'LDPE', 'Nylon', 'PBAT', 'PBS',
       'PC', 'PEEK', 'PEI', 'PET', 'PLA', 'PMMA', 'POM', 'PP', 'PS',
       'PTEE', 'PU', 'PVA', 'PVC', 'cellulose'], dtype='<U9')

In [25]:
unknown_dataset = UnknownSpectraDataset(data_dir)

Reading unknown data
Read 320 spectra for predicting.
No. of data points/shifts per spectra (input shape) : (1792,)


In [26]:
def predict_unknown_spectra(model, unknown_dataset, class_name):
    pred = model.predict(unknown_dataset.train_data)
    pred_bin = pred>=0.5
    res = []
    for p in pred_bin:
        res.append(class_name[p])
    return pred, pred_bin, res

In [27]:
pred, pred_bin, res = predict_unknown_spectra(ann, unknown_dataset, dataset.class_name)

10/10 [==============================] - 0s 1ms/step


In [29]:
for n, t in sorted(zip(unknown_dataset.fl, res)):
    if len(t) > 0:
        print(f"{os.path.basename(n):25s}, {t}")

1 mm-EFF2F-SB_1.csv      , ['PBS']
1 mm-EFF2F-SB_10.csv     , ['Acrylic']
1 mm-EFF2F-SB_4.csv      , ['Acrylic' 'PP' 'PVC']
1 mm-EFF2F-SB_5.csv      , ['PVC']
1 mm-EFF2F-SB_6.csv      , ['PVC']
1 mm-EFF2F-SB_7.csv      , ['PVC']
1 mm-EFF2F-SB_8.csv      , ['Acrylic' 'PVC']
1mm-1NF1G-SB_3.csv       , ['PET']
1mm-1NF1G-SB_4.csv       , ['PLA']
1mm-1NF1G-SB_5.csv       , ['PVC']
1mm-1NF1G-SB_6.csv       , ['PLA']
1mm-1NF1G-SB_7.csv       , ['PLA']
1mm-1NF1G-SB_8.csv       , ['PTEE']
1mm-1NF2F-SB_7.csv       , ['PVC']
1mm-B1-SB_8.csv          , ['PBS']
1mm-B2-SB_1.csv          , ['PLA']
1mm-B2-SB_13.csv         , ['PLA']
1mm-B2-SB_15.csv         , ['PBS']
1mm-B2-SB_16.csv         , ['PLA' 'PTEE']
1mm-B2-SB_17.csv         , ['PLA']
1mm-B2-SB_18.csv         , ['PLA']
1mm-B2-SB_19.csv         , ['PLA' 'PTEE']
1mm-B2-SB_2.csv          , ['PLA']
1mm-B2-SB_20.csv         , ['PLA']
1mm-B2-SB_22.csv         , ['PLA']
1mm-B2-SB_23.csv         , ['PLA']
1mm-B2-SB_25.csv         , ['PLA']
1mm-B2-SB_2

In [30]:
for t, n in zip(dataset.class_name, pred_bin.sum(axis=0)):
    print(f"{t:10s}: {n}")

Acrylic   : 5
ENR       : 0
EPDM      : 0
HDPE      : 0
LDPE      : 0
Nylon     : 0
PBAT      : 0
PBS       : 14
PC        : 1
PEEK      : 0
PEI       : 0
PET       : 1
PLA       : 62
PMMA      : 0
POM       : 0
PP        : 3
PS        : 0
PTEE      : 18
PU        : 0
PVA       : 0
PVC       : 36
cellulose : 3
